In [3]:
import pandas as pd

In [28]:
df = pd.read_csv('companies_pricing_data.csv',parse_dates=['DATE'],dayfirst=True,infer_datetime_format=True)
print(df.to_string())

            DATE  TICKER                  SECTOR      PRICEOPEN      PRICEHIGH       PRICELOW     PRICECLOSE     VOLUME
0     2021-01-04    AAPL  Information Technology     133.520000     133.611600     126.760000     129.410000  143301900
1     2021-01-05    AAPL  Information Technology     128.890000     131.740000     128.430000     131.010000   97664900
2     2021-01-06    AAPL  Information Technology     127.720000     131.049900     126.382000     126.600000  155088000
3     2021-01-07    AAPL  Information Technology     128.360000     131.630000     127.860000     130.920000  109578200
4     2021-01-08    AAPL  Information Technology     132.430000     132.630000     130.230000     132.050000  105158200
5     2021-01-11    AAPL  Information Technology     129.190000     130.170000     128.500000     128.980000  100620900
6     2021-01-12    AAPL  Information Technology     128.500000     129.690000     126.860000     128.800000   91951150
7     2021-01-13    AAPL  Information Te

In [410]:
pd.options.display.max_columns = None

In [5]:
PRICEAVG = []
for a,b in zip(df["PRICELOW"],df["PRICEHIGH"]):
    PRICEAVG.append((a+b)/2)
df["PRICEMID"] = PRICEAVG

In [6]:
SMA_5 = []
priceclose = []
ticker = ""
for a,b in zip(df['TICKER'],df['PRICECLOSE']):
    if ticker != a:
        ticker = a
        priceclose = []
    priceclose.append(b)
    if(len(priceclose)<5):
        SMA_5.append(None)
    elif(len(priceclose) == 6):
        priceclose = priceclose[1:]
    if(len(priceclose) == 5):
        SMA_5.append(sum(priceclose)/5)
df["SMA_5"] = SMA_5

In [7]:
dailyreturn = []
priceclose = 0
ticker = ""
for a,b in zip(df['TICKER'],df['PRICECLOSE']):
    if ticker != a:
        ticker = a
        priceclose = b
        dailyreturn.append(None)
    else:
        newpriceclose = b
        global value
        value = (newpriceclose/priceclose-1) * 100
        dailyreturn.append(str(value)+"%")
        priceclose = newpriceclose
df["DAILY_RETURN"] = dailyreturn

In [16]:
import datetime

In [9]:
def beforedate(currentdate,n):
    beforedate = currentdate - datetime.timedelta(days=n)
    return beforedate

def afterdate(currentdate,n):
    afterdate = currentdate + datetime.timedelta(days=n)
    return afterdate

def datecompare(a,b):
    result = (a-b).total_seconds()
    return result >= 0

In [416]:
print(afterdate(df.loc[0,'DATE'],30))
datecompare(df.loc[1,'DATE'],df.loc[2,'DATE'])

2021-02-03 00:00:00


False

In [ ]:
monthly = []
beforepriceclose = 0
ticker = ""
recordstartdate = ""

for date,b,a in zip(df["DATE"],df["PRICECLOSE"],df["TICKER"]):
    if ticker != a:
        ticker = a
        recordstartdate = afterdate(date,30)
        monthly.append(None)
        continue
    if datecompare(date,recordstartdate) == False:
        monthly.append(None)
        continue
    i = 30
    while True:
        date30daysbefore = beforedate(date,i)
        str_expr = "DATE == '{}' & TICKER == '{}'".format(date30daysbefore,ticker)
        df_result = df.query(str_expr)['PRICECLOSE'].values 
        if len(df_result) == 1:
            priceclose = df_result[0]
            newpriceclose = b
            value = (float(newpriceclose) - priceclose) / priceclose * 100
            monthly.append(str(value)+"%")
            break
        else:
            i+=1
df["MONTHLY_RETURN"] = monthly   

In [442]:
df.to_csv("companies_pricing_data_v3.csv")

In [419]:
df_std = df['PRICECLOSE'].std()
print(round(df_std))

55380


In [420]:
print(min(df['PRICELOW']))

7.2


In [430]:
dailyreturn = []
priceclose = 0
ticker = ""
for a,b in zip(df['TICKER'],df['PRICECLOSE']):
    if ticker != a:
        ticker = a
        priceclose = b
        dailyreturn.append(None)
    else:
        newpriceclose = b
        global value
        value = newpriceclose/priceclose-1
        dailyreturn.append(value)
        priceclose = newpriceclose
df["DAILY_RETURN"] = dailyreturn
df

,DATE,TICKER,SECTOR,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,PRICEMID,SMA_5,DAILY_RETURN,MONTHLY_RETURN
0,2021-01-04,AAPL,Information Technology,133.52,133.6116,126.760,129.41,143301900,130.18580,NaN,NaN,NaN
1,2021-01-05,AAPL,Information Technology,128.89,131.7400,128.430,131.01,97664900,130.08500,NaN,0.012364,NaN
2,2021-01-06,AAPL,Information Technology,127.72,131.0499,126.382,126.60,155088000,128.71595,NaN,-0.033662,NaN
3,2021-01-07,AAPL,Information Technology,128.36,131.6300,127.860,130.92,109578200,129.74500,NaN,0.034123,NaN
4,2021-01-08,AAPL,Information Technology,132.43,132.6300,130.230,132.05,105158200,131.43000,129.998,0.008631,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13659,2021-12-27,ZG,Real Estate,62.03,62.4199,60.750,60.99,796153,61.58495,60.650,-0.021655,10.749955
13660,2021-12-28,ZG,Real Estate,60.73,61.7500,59.300,60.00,772907,60.52500,60.982,-0.016232,8.952243
13661,2021-12-29,ZG,Real Estate,59.85,60.2000,58.560,59.21,352335,59.38000,60.668,-0.013167,9.324225
13662,2021-12-30,ZG,Real Estate,59.33,61.8700,58.800,61.27,752142,60.33500,60.762,0.034791,13.232305


In [431]:
df2 = df[((df['TICKER']=="BAC")|(df['TICKER']=="BRK.A")|(df['TICKER']=="CLST")|(df['TICKER']=="JPM")|(df['TICKER']=="WFC"))&(df['DATE'] <= '2021-06-30')]
df2

,DATE,TICKER,SECTOR,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,PRICEMID,SMA_5,DAILY_RETURN,MONTHLY_RETURN
1008,2021-01-04,BAC,Financials,30.46,30.570,29.7500,30.03,51483910,30.16000,NaN,NaN,NaN
1009,2021-01-05,BAC,Financials,30.17,30.540,29.9100,30.26,48507020,30.22500,NaN,0.007659,NaN
1010,2021-01-06,BAC,Financials,31.90,32.620,31.3550,32.15,107002800,31.98750,NaN,0.062459,NaN
1011,2021-01-07,BAC,Financials,33.31,33.620,32.8400,32.86,74358470,33.23000,NaN,0.022084,NaN
1012,2021-01-08,BAC,Financials,32.92,33.000,32.1131,32.53,44641360,32.55655,31.566,-0.010043,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12775,2021-06-24,WFC,Financials,44.26,45.420,44.2100,45.18,27951010,44.81500,43.542,0.027753,-1.974398
12776,2021-06-25,WFC,Financials,45.57,46.650,45.2200,46.38,49074750,45.93500,44.468,0.026560,-0.043103
12777,2021-06-28,WFC,Financials,46.00,46.070,45.0900,45.79,27127540,45.58000,44.970,-0.012721,-1.990582
12778,2021-06-29,WFC,Financials,46.04,46.380,44.6000,44.79,39534030,45.49000,45.220,-0.021839,-4.130993


In [432]:
df2.loc[df2[['DAILY_RETURN']].idxmin()]

,DATE,TICKER,SECTOR,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,PRICEMID,SMA_5,DAILY_RETURN,MONTHLY_RETURN
12665,2021-01-15,WFC,Financials,33.08,33.5,31.82,32.04,91993100,32.66,33.554,-0.077986,NaN


In [26]:
monthly = []
beforepriceclose = 0
ticker = ""
recordstartdate = ""

for date,b,a in zip(df["DATE"],df["PRICECLOSE"],df["TICKER"]):
    if ticker != a:
        ticker = a
        recordstartdate = afterdate(date,30)
        monthly.append(None)
        continue
    if datecompare(date,recordstartdate) == False:
        monthly.append(None)
        continue
    i = 30
    while True:
        date30daysbefore = beforedate(date,i)
        str_expr = "DATE == '{}' & TICKER == '{}'".format(date30daysbefore,ticker)
        df_result = df.query(str_expr)['PRICECLOSE'].values 
        if len(df_result) == 1:
            priceclose = df_result[0]
            newpriceclose = b
            value = (float(newpriceclose) - priceclose) / priceclose * 100
            monthly.append(value)
            break
        else:
            i+=1
df["MONTHLY_RETURN"] = monthly  
df

,DATE,TICKER,SECTOR,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,MONTHLY_RETURN
0,2021-01-04,AAPL,Information Technology,133.52,133.6116,126.760,129.41,143301900,NaN
1,2021-01-05,AAPL,Information Technology,128.89,131.7400,128.430,131.01,97664900,NaN
2,2021-01-06,AAPL,Information Technology,127.72,131.0499,126.382,126.60,155088000,NaN
3,2021-01-07,AAPL,Information Technology,128.36,131.6300,127.860,130.92,109578200,NaN
4,2021-01-08,AAPL,Information Technology,132.43,132.6300,130.230,132.05,105158200,NaN
...,...,...,...,...,...,...,...,...,...
13659,2021-12-27,ZG,Real Estate,62.03,62.4199,60.750,60.99,796153,10.749955
13660,2021-12-28,ZG,Real Estate,60.73,61.7500,59.300,60.00,772907,8.952243
13661,2021-12-29,ZG,Real Estate,59.85,60.2000,58.560,59.21,352335,9.324225
13662,2021-12-30,ZG,Real Estate,59.33,61.8700,58.800,61.27,752142,13.232305


In [433]:
df3 = df[((df['TICKER']=="AAPL")|(df['TICKER']=="MA")|(df['TICKER']=="MSFT")|(df['TICKER']=="NVDA")|(df['TICKER']=="V"))]

In [434]:
df.loc[df[['MONTHLY_RETURN']].idxmin()]

,DATE,TICKER,SECTOR,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,PRICEMID,SMA_5,DAILY_RETURN,MONTHLY_RETURN
13641,2021-11-30,ZG,Real Estate,53.8,55.47,52.49,54.11,1255504,53.98,54.768,-0.000923,-48.817631


In [435]:
df.groupby(['SECTOR']).mean()

,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,PRICEMID,SMA_5,DAILY_RETURN,MONTHLY_RETURN
SECTOR,,,,,,,,,
Communication Services,718.693053,725.661935,711.612090,718.915611,1.038514e+07,718.637013,719.513487,0.000737,1.438209
Consumer Discretionary,331.648151,336.502220,326.781382,331.755249,2.160895e+07,331.641801,331.287875,0.001220,2.285924
Consumer Staples,181.029057,182.442157,179.752727,181.160940,7.777836e+06,181.097442,180.918714,0.000737,1.697918
Energy,91.261188,92.557745,90.020553,91.310817,1.006686e+07,91.289149,91.384324,0.002131,4.035382
Financials,97398.454260,98033.632450,96718.770643,97362.141495,2.137804e+07,97376.201547,97702.176476,0.001423,2.869016
Health Care,273.394849,276.093290,270.870422,273.482032,9.593866e+06,273.481856,273.269729,0.001533,2.720869
Industrials,187.591439,189.428756,185.732157,187.521563,5.325484e+06,187.580456,187.616553,0.000690,1.350394
Information Technology,238.402392,240.995625,235.666759,238.383363,3.306289e+07,238.331192,238.302727,0.001435,3.204929
Materials,223.333784,225.262234,221.433283,223.361799,5.293180e+06,223.347758,223.242841,0.001213,2.120292


In [436]:
df.loc[df[['MONTHLY_RETURN']].idxmax()]

,DATE,TICKER,SECTOR,PRICEOPEN,PRICEHIGH,PRICELOW,PRICECLOSE,VOLUME,PRICEMID,SMA_5,DAILY_RETURN,MONTHLY_RETURN
11356,2021-11-04,TSLA,Consumer Discretionary,1234.41,1243.49,1217.0,1229.91,25397410,1230.245,1187.672,0.013222,57.561588
